In [1]:
# load the numpy library with alias name "np"
import numpy as np
# load the pandas library with alias name "pd"
import pandas as pd

In [2]:
# Supress scientific notation using following command
pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
# Read the file using read_excel()
# use sheet_name to read specific worksheet from MS Excel file
# use index_col to set the index column for data frame
# use skiprows to skip any number of rows in the MS Excel file
nse_data = pd.read_excel('C:/Users/user/Documents/GitHub/AI-ML-Algorithms-for-Business-Applications/Datasets/NSE Stocks 22-Nov-2021.xlsx', skiprows= 5) 
nse_data.columns = nse_data.columns.str.lower().str.replace(' ','').str.replace('/','_')

#rename columns
nse_data = nse_data.rename(
     columns= {
         "companyname" : "company",
         "openingprice" : "open",
         "highprice" : "high",
         "lowprice" : "low",
         "closingprice" : "close", 
         "adjustedclosingprice": "close_adj", 
         "marketcapitalisation" : "marketcap"
     }
)

nse_data.head()

,company,beta,alpha,niftyclosing,date,open,high,low,close,close_adj,sharesoutstanding,marketcap,totalreturns,facevalue,eps,p_e,p_b,yield
0,20 Microns Ltd.,1.23,-0.08,"17,416.55",2021-11-22,60.25,61.10,56.25,57.50,57.50,"35,286,502.00","2,028.97",-4.56,5.00,10.70,5.37,0.96,0.00
1,20Th Century Finance Corpn. Ltd. [Merged],0.61,-0.61,"1,160.15",1999-05-18,12.50,12.50,12.10,12.50,12.50,"17,470,191.00",218.38,-0.79,10.00,9.63,1.30,0.12,19.20
2,3I Infotech Ltd.,1.49,0.18,"17,416.55",2021-11-22,89.35,89.35,89.35,89.35,89.35,"164,036,287.00","14,656.64",4.99,10.00,1.45,61.66,NaN,0.00
3,3M India Ltd.,0.90,0.21,"17,416.55",2021-11-22,"25,200.00","25,600.00","25,001.00","25,250.90","25,250.90","11,265,070.00","284,453.16",0.23,10.00,185.40,136.20,12.29,0.00
4,3P Land Holdings Ltd.,0.92,0.09,"17,416.55",2021-11-22,13.95,13.95,12.75,12.85,12.85,"18,000,000.00",231.30,-4.10,2.00,0.02,771.00,0.47,0.00


In [4]:
# Calculate total number of shares using market cap divided by closing price
# Note: athematical operators (+, -, \*, /) or logical operators (<, >, =,...) work element wise. 
nse_data["total_stocks"] = (nse_data["marketcap"])/(nse_data["close_adj"])
nse_data[{'sharesoutstanding', 'total_stocks'}]

,total_stocks,sharesoutstanding
0,35.29,"35,286,502.00"
1,17.47,"17,470,191.00"
2,164.04,"164,036,287.00"
3,11.27,"11,265,070.00"
4,18.00,"18,000,000.00"
...,...,...
6574,NaN,NaN
6575,29.44,"29,440,604.00"
6576,NaN,NaN
6577,63.63,"63,632,144.00"


## Create groups of companies based on market cap
* Small cap : marketcap < 5000 Crs
* Mid cap : 5000 Crs < marketcap 20000 Crs
* Large cap : marketcap > 20000 Crs
* Create invalid group if marketcap is unavailable

In [5]:
# While multiple techniques exist to create a separate categorical column, the most efficient method is using np.select()
# Reference 1: https://towardsdatascience.com/the-most-efficient-if-elif-else-in-pandas-d4a7d4502d50
# Reference 2: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

# create condition for company_type

conditions =  [
    (nse_data['marketcap'] <= 5000),
    (nse_data['marketcap'] > 5000) & (nse_data['marketcap'] <= 20000),
    (nse_data['marketcap'] > 20000), 
    (nse_data['marketcap'].isna()) # for companies with invalid marketcap values
        ]

# Create company type values
cap_values = ['small-cap', 'mid-cap', 'large-cap', 'invalid']

# Create a new column company_type in nse_data
nse_data['company_type'] = np.select(conditions, cap_values)

# View the details of company, marketcap, & company_type
nse_data[{'company', 'marketcap', 'company_type'}]


,marketcap,company_type,company
0,"2,028.97",small-cap,20 Microns Ltd.
1,218.38,small-cap,20Th Century Finance Corpn. Ltd. [Merged]
2,"14,656.64",mid-cap,3I Infotech Ltd.
3,"284,453.16",large-cap,3M India Ltd.
4,231.30,small-cap,3P Land Holdings Ltd.
...,...,...,...
6574,NaN,invalid,Zuari Forex Ltd. (1988) [Merged]
6575,"3,984.79",small-cap,Zuari Global Ltd.
6576,NaN,invalid,Zyden Gentec Ltd.
6577,"128,272.86",large-cap,Zydus Wellness Ltd.


In [6]:
# Count number of NaN values in marketcap column
nse_data['marketcap'].isna().sum()


3390

In [7]:
# View the companies with marketcap as NaN
nse_data[nse_data['marketcap'].isna()]

,company,beta,alpha,niftyclosing,date,open,high,low,close,close_adj,sharesoutstanding,marketcap,totalreturns,facevalue,eps,p_e,p_b,yield,total_stocks,company_type
6,52 Weeks Entertainment Ltd.,0.05,-1.71,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
9,7Nr Retail Ltd.,0.10,-0.15,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
10,7Seas Entertainment Ltd.,-0.07,0.32,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
11,A & M Febcon Ltd.,0.36,-0.69,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
15,A A R V Infratel Ltd.,1.59,2.65,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,Zim Laboratories Ltd.,0.18,0.15,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
6568,Zodiac Ventures Ltd.,0.55,-2.22,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
6571,Zora Pharma Ltd. [Merged],0.91,0.82,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
6574,Zuari Forex Ltd. (1988) [Merged],0.34,1.21,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid


In [8]:
# Group by Categorical
# use group_by() method followed by aggregate methods

# Fetch the number of companies based on company type (market cap)

# Fetches the count of all numeric data type columns
nse_data.groupby(['company_type']).count()


,company,beta,alpha,niftyclosing,date,open,high,low,close,close_adj,sharesoutstanding,marketcap,totalreturns,facevalue,eps,p_e,p_b,yield,total_stocks
company_type,,,,,,,,,,,,,,,,,,,
invalid,3390,3385,3385,6,6,6,6,6,6,6,0,0,6,0,0,0,0,0,0
large-cap,766,686,686,766,766,766,766,766,766,766,766,766,766,766,760,655,740,766,766
mid-cap,446,425,425,446,446,446,446,446,446,446,446,446,446,446,445,350,431,446,446
small-cap,1977,1810,1810,1977,1977,1977,1977,1977,1977,1977,1969,1977,1977,1969,1776,840,1476,1977,1977


In [9]:
# Fetches the count of all numeric data type columns
nse_data.groupby(['company_type']).count()['company']

company_type
invalid      3390
large-cap     766
mid-cap       446
small-cap    1977
Name: company, dtype: int64

In [10]:
# Create another pandas dataframe with companies having invalid marketcap
# delete them from nse_data dataframe
# use isna() method. Another way is to use conditions
nse_invalid = nse_data[nse_data['marketcap'].isna()]
nse_invalid.head()

,company,beta,alpha,niftyclosing,date,open,high,low,close,close_adj,sharesoutstanding,marketcap,totalreturns,facevalue,eps,p_e,p_b,yield,total_stocks,company_type
6,52 Weeks Entertainment Ltd.,0.05,-1.71,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
9,7Nr Retail Ltd.,0.10,-0.15,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
10,7Seas Entertainment Ltd.,-0.07,0.32,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
11,A & M Febcon Ltd.,0.36,-0.69,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid
15,A A R V Infratel Ltd.,1.59,2.65,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,invalid


In [11]:
# delete companies having invalid marketcap from nse_data dataframe
# use notna() method. Another way is to use conditions
nse_data = nse_data[nse_data['marketcap'].notna()]
nse_data.head()

,company,beta,alpha,niftyclosing,date,open,high,low,close,close_adj,sharesoutstanding,marketcap,totalreturns,facevalue,eps,p_e,p_b,yield,total_stocks,company_type
0,20 Microns Ltd.,1.23,-0.08,"17,416.55",2021-11-22,60.25,61.10,56.25,57.50,57.50,"35,286,502.00","2,028.97",-4.56,5.00,10.70,5.37,0.96,0.00,35.29,small-cap
1,20Th Century Finance Corpn. Ltd. [Merged],0.61,-0.61,"1,160.15",1999-05-18,12.50,12.50,12.10,12.50,12.50,"17,470,191.00",218.38,-0.79,10.00,9.63,1.30,0.12,19.20,17.47,small-cap
2,3I Infotech Ltd.,1.49,0.18,"17,416.55",2021-11-22,89.35,89.35,89.35,89.35,89.35,"164,036,287.00","14,656.64",4.99,10.00,1.45,61.66,NaN,0.00,164.04,mid-cap
3,3M India Ltd.,0.90,0.21,"17,416.55",2021-11-22,"25,200.00","25,600.00","25,001.00","25,250.90","25,250.90","11,265,070.00","284,453.16",0.23,10.00,185.40,136.20,12.29,0.00,11.27,large-cap
4,3P Land Holdings Ltd.,0.92,0.09,"17,416.55",2021-11-22,13.95,13.95,12.75,12.85,12.85,"18,000,000.00",231.30,-4.10,2.00,0.02,771.00,0.47,0.00,18.00,small-cap


## Grouping based on categories
* use group_by() method in pandas
* Group happens in this pattern: split, apply & combine
* Split the data into groups
* Apply a function to each group independently
* Combine the results into a data structure

In [12]:
# Group by the company type
nse_data.groupby(['company_type']).describe()

beta                                            \
                            count                 mean                  std   
company_type                                                                  
large-cap                  686.00                 1.15                 0.39   
mid-cap                    425.00                 1.26                 0.37   
small-cap                1,810.00                 1.00                 0.59   

                                                                             \
                              min                  25%                  50%   
company_type                                                                  
large-cap                   -0.55                 0.87                 1.13   
mid-cap                      0.15                 1.05                 1.28   
small-cap                   -8.27                 0.66                 1.02   

                                                                      alpha  \
                              75%                  max                count   
company_type                                                                  
large-cap                    1.42                 2.35               686.00   
mid-cap                      1.50                 2.25               425.00   
small-cap                    1.34                 4.58             1,810.00   

                                   ...                yield  \
                             mean  ...                  75%   
company_type                       ...                        
large-cap                    0.15  ...                 1.07   
mid-cap                      0.05  ...                 0.83   
small-cap                    0.08  ...                 0.00   

                                          total_stocks                       \
                              max                count                 mean   
company_type                                                                  
large-cap                   21.23               766.00               895.93   
mid-cap                     13.30               446.00               328.19   
small-cap                  667.35             1,977.00                66.32   

                                                                             \
                              std                  min                  25%   
company_type                                                                  
large-cap                2,289.09                 2.57                73.01   
mid-cap                  3,478.51                 2.00                21.03   
small-cap                  246.09                 0.00                10.20   

                                                                             
                              50%                  75%                  max  
company_type                                                                 
large-cap                  187.76               621.37            28,735.39  
mid-cap                     46.39               127.09            72,136.60  
small-cap                   18.19                41.97             6,608.10  

[3 rows x 136 columns]

In [13]:
# In case you are interested in two columns, marketcap and close_adj
nse_data.groupby(['company_type']).describe()[{'marketcap', 'close_adj'}]

close_adj                                            \
                            count                 mean                  std   
company_type                                                                  
large-cap                  766.00             1,471.51             4,097.83   
mid-cap                    446.00               379.02               482.24   
small-cap                1,977.00                54.40               134.95   

                                                                             \
                              min                  25%                  50%   
company_type                                                                  
large-cap                    4.20               195.50               544.83   
mid-cap                      0.10                80.14               219.00   
small-cap                    0.05                 2.85                11.35   

                                                                  marketcap  \
                              75%                  max                count   
company_type                                                                  
large-cap                1,388.61            75,657.60               766.00   
mid-cap                    507.55             4,702.35               446.00   
small-cap                   52.50             2,400.00             1,977.00   

                                                                             \
                             mean                  std                  min   
company_type                                                                  
large-cap              341,683.50         1,023,431.34            20,105.59   
mid-cap                 10,990.70             4,363.71             5,017.99   
small-cap                  847.70             1,176.45                 0.00   

                                                                             \
                              25%                  50%                  75%   
company_type                                                                  
large-cap               38,247.45            82,988.71           248,948.88   
mid-cap                  7,156.55            10,111.99            14,508.23   
small-cap                   66.18               258.37             1,188.91   

                                   
                              max  
company_type                       
large-cap           15,993,061.62  
mid-cap                 19,958.00  
small-cap                4,923.45

In [19]:
# when you are only interested in mean of marketcap & close_adj and not other attributes
nse_data.groupby(['company_type']).mean()[{'marketcap', 'close_adj'}]

,close_adj,marketcap
company_type,,
large-cap,"1,471.51","341,683.50"
mid-cap,379.02,"10,990.70"
small-cap,54.40,847.70


In [20]:
# another way to do the same thing as above
# using agg method
nse_data.groupby(['company_type']).agg('mean')[{'marketcap', 'close_adj'}]

,close_adj,marketcap
company_type,,
large-cap,"1,471.51","341,683.50"
mid-cap,379.02,"10,990.70"
small-cap,54.40,847.70


In [21]:
# you may use agg() method do get more than mean
nse_data.groupby(['company_type']).agg(['mean','count','sum','min','max'])[{'marketcap', 'close_adj'}]


close_adj                             \
                             mean count                  sum   
company_type                                                   
large-cap                1,471.51   766         1,127,172.90   
mid-cap                    379.02   446           169,041.65   
small-cap                   54.40  1977           107,552.96   

                                                                  marketcap  \
                              min                  max                 mean   
company_type                                                                  
large-cap                    4.20            75,657.60           341,683.50   
mid-cap                      0.10             4,702.35            10,990.70   
small-cap                    0.05             2,400.00               847.70   

                                                              \
             count                  sum                  min   
company_type                                                   
large-cap      766       261,729,564.61            20,105.59   
mid-cap        446         4,901,854.39             5,017.99   
small-cap     1977         1,675,902.28                 0.00   

                                   
                              max  
company_type                       
large-cap           15,993,061.62  
mid-cap                 19,958.00  
small-cap                4,923.45

In [14]:
# Fetch the count by company_type
nse_data.groupby(['company_type'])['company_type'].count()

company_type
large-cap     766
mid-cap       446
small-cap    1977
Name: company_type, dtype: int64

In [15]:
# Fetch the count by company_type
# same as above cell
nse_data['company_type'].value_counts()

small-cap    1977
large-cap     766
mid-cap       446
Name: company_type, dtype: int64

In [16]:
# aggregate NSE data by companies with highest market cap
nse_data.sort_values(by = "marketcap")

# the above line of code values are in ascending order. 
# change to descending order
nse_data.sort_values(by = "marketcap", ascending= False)

,company,beta,alpha,niftyclosing,date,open,high,low,close,close_adj,sharesoutstanding,marketcap,totalreturns,facevalue,eps,p_e,p_b,yield,total_stocks,company_type
4734,Reliance Industries Ltd.,0.92,0.35,"17,416.55",2021-11-22,"2,436.10","2,449.00","2,351.00","2,363.75","2,363.75","6,765,970,014.00","15,993,061.62",-4.43,10.00,57.38,41.20,3.57,0.30,"6,765.97",large-cap
5859,Tata Consultancy Services Ltd.,0.54,0.30,"17,416.55",2021-11-22,"3,478.40","3,515.95","3,440.30","3,458.40","3,458.40","3,699,051,373.00","12,792,799.27",-0.50,1.00,96.17,35.96,15.51,1.01,"3,699.05",large-cap
2185,H D F C Bank Ltd.,0.90,0.21,"17,416.55",2021-11-22,"1,546.00","1,552.70","1,499.05","1,515.35","1,515.35","5,540,421,536.00","8,395,677.77",-1.56,1.00,60.36,25.11,3.87,0.43,"5,540.42",large-cap
2606,Infosys Ltd.,0.55,0.27,"17,416.55",2021-11-22,"1,784.00","1,785.40","1,751.15","1,759.40","1,759.40","4,205,539,009.00","7,399,225.33",-1.12,5.00,46.91,37.50,12.37,1.71,"4,205.54",large-cap
2335,Hindustan Unilever Ltd.,0.52,0.29,"17,416.55",2021-11-22,"2,409.40","2,409.90","2,377.15","2,388.55","2,388.55","2,349,591,262.00","5,612,116.21",-0.45,1.00,34.93,68.38,12.71,1.34,"2,349.59",large-cap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,Embassy Office Parks Reit,0.59,-0.06,"17,416.55",2021-11-22,367.00,375.45,365.80,368.01,368.01,NaN,0.00,0.37,NaN,NaN,0.00,0.00,0.00,0.00,small-cap
6500,Wisec Global Ltd.,-0.55,0.58,948.20,1997-02-03,36.00,36.00,36.00,36.00,36.00,NaN,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,small-cap
935,Brookfield India Real Estate Trust,NaN,NaN,"17,416.55",2021-11-22,299.80,303.00,287.50,296.95,296.95,NaN,0.00,-0.03,NaN,NaN,0.00,0.00,0.00,0.00,small-cap
5026,Sandoz (India) Ltd. [Merged],NaN,NaN,985.30,1996-03-29,385.00,385.00,384.00,384.00,384.00,NaN,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,small-cap


In [18]:
#When you
nse_data.groupby(['company_type']).mean()[{'marketcap', 'close_adj'}]

,close_adj,marketcap
company_type,,
large-cap,"1,471.51","341,683.50"
mid-cap,379.02,"10,990.70"
small-cap,54.40,847.70
